In [4]:
import src.models.helpers as mh
import src.models.builders as mb
import src.inference.helpers as ih
import src.data.breathe_data as breathe_data
import src.data.smartcare_data as smartcare_data
import src.data.helpers as dh

import numpy as np
import pandas as pd

In [5]:
df = breathe_data.load_o2_fev1_df_from_excel()
# df = smartcare_data.load_o2_fev1_df_from_excel()
df.head()

,ID,Date Recorded,FEV1,O2 Saturation,ecFEV1,Age,Sex,Height,Predicted FEV1,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy
0,101,2019-02-20,1.31,97.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,36.287474,99.767593
1,101,2019-02-21,1.29,96.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,35.733466,98.739061
2,101,2019-02-22,1.32,96.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,36.564477,98.739061
3,101,2019-02-23,1.28,97.0,1.33,53,Male,173.0,3.610061,97.22596,36.841481,35.456463,99.767593
4,101,2019-02-24,1.33,98.0,1.36,53,Male,173.0,3.610061,97.22596,37.672492,36.841481,100.796125


In [7]:
# Infer AR and IA for all data points given an individuals' age, sex, height, FEV1 and O2 saturation measurements
def infer_AR_IA_for_ID(df):
    df.reset_index(inplace=True)
    _, inf_alg, _, ecFEV1, AR, _, _, IA, _, O2Sat = mb.o2sat_fev1_point_in_time_model(
        df.Height[0], df.Age[0], df.Sex[0]
    )
    # _, inf_alg, AR, IA, ecFEV1, O2Sat = build_model(173, 53, "Male")

    # inf_res = ih.infer(inf_alg, [AR, IA], [[ecFEV1, 1.31], [O2Sat, 97.0]], show_progress=False, joint=False, get_time=True)

    def infer_and_unpack(ecfev1_obs, o2sat_obs):
        res = ih.infer(
            inf_alg,
            [AR, IA],
            [[ecFEV1, ecfev1_obs], [O2Sat, o2sat_obs]],
            show_progress=False,
            joint=False,
            get_time=False,
        )
        return res[AR.name].values, res[IA.name].values

    res = df.apply(
        lambda row: infer_and_unpack(row["ecFEV1"], row["O2 Saturation"]),
        axis=1,
    )
    return res


resraw = df.groupby("ID").apply(infer_AR_IA_for_ID)
# resraw = df.iloc[np.r_[10:13, 1000:1007]].groupby("ID").apply(infer_AR_IA_for_ID)
res = (
    resraw.apply(pd.Series)
    .reset_index()
    .rename(columns={0: "AR", 1: "IA"})
    .drop(columns="level_1")
)
res

/Applications/anaconda3/envs/phd/lib/python3.10/site-packages/pgmpy/factors/discrete/DiscreteFactor.py:801: RuntimeWarning: invalid value encountered in divide
  phi.values = phi.values / phi1.values
/Applications/anaconda3/envs/phd/lib/python3.10/site-packages/pgmpy/factors/discrete/DiscreteFactor.py:801: RuntimeWarning: invalid value encountered in divide
  phi.values = phi.values / phi1.values
/Applications/anaconda3/envs/phd/lib/python3.10/site-packages/pgmpy/factors/discrete/DiscreteFactor.py:801: RuntimeWarning: invalid value encountered in divide
  phi.values = phi.values / phi1.values
/Applications/anaconda3/envs/phd/lib/python3.10/site-packages/pgmpy/factors/discrete/DiscreteFactor.py:801: RuntimeWarning: invalid value encountered in divide
  phi.values = phi.values / phi1.values
/Applications/anaconda3/envs/phd/lib/python3.10/site-packages/pgmpy/factors/discrete/DiscreteFactor.py:801: RuntimeWarning: invalid value encountered in divide
  phi.values = phi.values / phi1.values


,ID,AR,IA
0,101,"[4.325885612867902e-05, 5.204186795805041e-05,...","[0.7890913910456894, 0.18731794516480815, 0.02..."
1,101,"[2.412546137918627e-05, 2.9023746531528923e-05...","[0.6220562020672772, 0.2968488819658886, 0.071..."
2,101,"[2.412546137918627e-05, 2.9023746531528923e-05...","[0.6220562020672772, 0.2968488819658886, 0.071..."
3,101,"[4.325885612867902e-05, 5.204186795805041e-05,...","[0.7890913910456894, 0.18731794516480815, 0.02..."
4,101,"[9.946039499055924e-05, 0.00012030272039134185...","[0.8844314501002895, 0.10899967575332596, 0.00..."
...,...,...,...
20392,358,"[0.12473481300803844, 0.1290133320347184, 0.12...","[0.9008728220064204, 0.09561009435187642, 0.00..."
20393,358,"[0.12473481300803844, 0.1290133320347184, 0.12...","[0.9008728220064204, 0.09561009435187642, 0.00..."
20394,358,"[0.12473481300803846, 0.1290133320347184, 0.12...","[0.7940361825789909, 0.185494556181832, 0.0195..."
20395,358,"[0.11434194793175263, 0.12023910500669462, 0.1...","[0.6088054139470103, 0.31077666011484656, 0.07..."


In [8]:
AR = mh.variableNode("Airway resistance (%)", 0, 90, 2, prior={"type": "uniform"})
IA = mh.variableNode("Inactive alveoli (%)", 0, 30, 1, prior={"type": "uniform"})

res["AR mean"] = res["AR"].apply(lambda x: AR.get_mean(x))
res["IA mean"] = res["IA"].apply(lambda x: IA.get_mean(x))

In [9]:
# Concatenate the 4 new columns to the original dataframe
df = pd.concat([df, res.drop(columns=["ID"])], axis=1)
df.head()

,ID,Date Recorded,FEV1,O2 Saturation,ecFEV1,Age,Sex,Height,Predicted FEV1,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy,AR,IA,AR mean,IA mean
0,101,2019-02-20,1.31,97.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,36.287474,99.767593,"[4.325885612867902e-05, 5.204186795805041e-05,...","[0.7890913910456894, 0.18731794516480815, 0.02...",58.502814,0.736011
1,101,2019-02-21,1.29,96.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,35.733466,98.739061,"[2.412546137918627e-05, 2.9023746531528923e-05...","[0.6220562020672772, 0.2968488819658886, 0.071...",59.743958,0.968994
2,101,2019-02-22,1.32,96.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,36.564477,98.739061,"[2.412546137918627e-05, 2.9023746531528923e-05...","[0.6220562020672772, 0.2968488819658886, 0.071...",59.743958,0.968994
3,101,2019-02-23,1.28,97.0,1.33,53,Male,173.0,3.610061,97.22596,36.841481,35.456463,99.767593,"[4.325885612867902e-05, 5.204186795805041e-05,...","[0.7890913910456894, 0.18731794516480815, 0.02...",58.502814,0.736011
4,101,2019-02-24,1.33,98.0,1.36,53,Male,173.0,3.610061,97.22596,37.672492,36.841481,100.796125,"[9.946039499055924e-05, 0.00012030272039134185...","[0.8844314501002895, 0.10899967575332596, 0.00...",55.679250,0.622275


In [11]:
# Save rawres as excel to ../../../../ExcelFiles
df.to_excel(
    dh.get_path_to_main()
    + "ExcelFiles/BR/inferred_AR_IA_full_model.xlsx",
    index=False,
)

# Save arrays as objects
Arrays are currently saved as strings which is painful to process when reading from the file

In [ ]:
import pandas as pd
import numpy as np
import src.data.helpers as data_helpers

In [ ]:
data_helpers.load_excel(
    "../../../../ExcelFiles/inferred_AR_IA_with_FEV1_O2Sat_no_AR-IA_factor_1.xlsx",
    ["AR", "IA"],
).iloc[0].AR

In [ ]:
df = pd.read_excel(
    "../../../../ExcelFiles/inferred_AR_IA_with_FEV1_O2Sat_no_AR-IA_factor_1.xlsx"
)

In [ ]:
# Convert back to arrays
def str_to_array(s):
    s_cleaned = s.replace("\\n", "")
    return np.fromstring(s_cleaned[1:-1], sep=" ")


df["AR"] = df.AR.apply(str_to_array)
df["IA"] = df.IA.apply(str_to_array)

In [ ]:
df.iloc[0].AR

In [ ]:
df.to_excel(
    "../../../../ExcelFiles/inferred_AR_IA_with_FEV1_O2Sat_no_AR-IA_factor_1.xlsx",
    index=False,
)

In [ ]:
df = pd.read_excel(
    "../../../../ExcelFiles/inferred_AR_IA_with_FEV1_O2Sat_no_AR-IA_factor.xlsx"
)

In [ ]:
df.AR.apply(json.loads)[0]